# Merge evaluations and obtain averaged results

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
import openpyxl
%matplotlib inline
import os

from warnings import filterwarnings
filterwarnings('ignore')

### 5 random state için sonuçların ortalamasını al ve yeni dosyaya bas

In [3]:
evaluation_dir = "./evaluation/"
output_file = "./results_averaged_std.xlsx"

# all dataset names
datasets = ['evaluation']


# rest of the code
 
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for data in datasets:
        # Read and concatenate Excel files
        dfs = []
        states = [1, 42, 123, 1234, 12345]
        for idx in states:
            filename = '{}_RS{}.xlsx'.format(data, idx)
            df = pd.read_excel(evaluation_dir + filename)
            dfs.append(df)

        # Concatenate all files
        merged_df = pd.concat(dfs, axis=0)

        # Calculate mean values
        averages = merged_df.groupby('Models').mean()
        averages = averages.drop(['Unnamed: 0'], axis=1)

        # Calculate standard deviation values
        std_devs = merged_df.groupby('Models').std()
        std_devs = std_devs.drop(['Unnamed: 0'], axis=1)

        # Combine averages and std_devs into one DataFrame and format the result
        result = averages.copy()
        for column in averages.columns:
            result[column] = (averages[column] * 100).map("{:.2f}".format) + " ± " + (std_devs[column] * 100).map("{:.2f}".format)

        

        # Write the result to the Excel file
        result.to_excel(writer, sheet_name=data)

        # Access worksheet and adjust formatting
        workbook = writer.book
        worksheet = writer.sheets[data]
        for col in worksheet.columns:
            max_length = 0
            column = col[0].column_letter
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
            worksheet.column_dimensions[column].width = adjusted_width
            for cell in col:
                cell.alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)